In [80]:
import numpy as np
import pandas as pd

import os
import cv2
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from skimage.feature import local_binary_pattern, hog
from skimage.filters import gabor

import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing Images

In [ ]:
def preprocess_image(image_path, output_path, size=(256, 256)):
    image = cv2.imread(image_path)

    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Remove noise using GaussianBlur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Resize the image
    resized_image = cv2.resize(blurred_image, size)

    # Save the processed image
    cv2.imwrite(output_path, resized_image)


In [ ]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/leaf-classification/leaves'
output_dir = '/content/drive/MyDrive/Colab Notebooks/leaf-classification/leaves-processed'
os.makedirs(output_dir, exist_ok=True)


for folder_name in os.listdir(root_dir):
  folder_path = os.path.join(root_dir, folder_name)

  if os.path.isdir(folder_path):
    output_folder_path = os.path.join(output_dir, folder_name)
    os.makedirs(output_folder_path, exist_ok=True)

    for img_name in os.listdir(folder_path):
      img_path = os.path.join(folder_path, img_name)
      output_img_path = os.path.join(output_folder_path, img_name)
      # process and save the image
      preprocess_image(img_path, output_img_path)


# Feature Extraction

In [ ]:
def extract_lbp_features(image):
    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist


def extract_hog_features(image):
    features, _ = hog(image, orientations=9, pixels_per_cell=(16, 16),
                      cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
    return features


def extract_orb_features(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros((64,))
    return descriptors.flatten()[:64]


def extract_gabor_features(image):
    filters = []
    num_scales = 5
    num_orientations = 8
    for theta in np.arange(0, np.pi, np.pi / num_orientations):
        for sigma in (1, 3):
            for frequency in (0.05, 0.25):
                kernel = cv2.getGaborKernel((21, 21), sigma, theta, frequency, 0.5, 0, ktype=cv2.CV_32F)
                filters.append(kernel)

    features = []
    for kernel in filters:
        fimg = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        features.extend(fimg.ravel())

    return np.array(features[:128])


In [ ]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/leaf-classification/leaves-processed'

features_list = []
folder_index = 0

for folder_name in os.listdir(root_dir):
  folder_index += 1
  folder_path = os.path.join(root_dir, folder_name)

  if os.path.isdir(folder_path):
    image_index = 0

    for img_name in os.listdir(folder_path):
      image_index += 1
      img_path = os.path.join(folder_path, img_name)
      image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

      lbf_features = extract_lbp_features(image)
      hog_features = extract_hog_features(image)
      #orb_features = extract_orb_features(image)
      #gabor_features = extract_gabor_features(image)

      extracted_features = np.concatenate([lbf_features, hog_features])
      features_list.append([folder_index, image_index] + extracted_features.tolist())



output_csv_path = '/content/drive/MyDrive/Colab Notebooks/leaf-classification/features.csv'
features_df = pd.DataFrame(features_list)
features_df.to_csv(output_csv_path, index=False)


In [ ]:
print(features_df.shape)
features_df.head()

(443, 8111)


,0,1,2,3,4,5,6,7,8,9,...,8101,8102,8103,8104,8105,8106,8107,8108,8109,8110
0,1,1,0.020538,0.068832,0.022583,0.104324,0.109726,0.174591,0.107117,0.097046,...,0.113195,0.229852,0.064980,0.065598,0.143267,0.229852,0.203198,0.202729,0.057279,0.068118
1,1,2,0.027725,0.079773,0.032181,0.103241,0.107132,0.151291,0.112457,0.098572,...,0.118712,0.214563,0.084731,0.161559,0.168356,0.214563,0.199051,0.144328,0.081321,0.039529
2,1,3,0.019913,0.071503,0.021408,0.100754,0.101318,0.175217,0.103973,0.099701,...,0.008018,0.200710,0.022678,0.158478,0.156798,0.230509,0.133093,0.101094,0.089969,0.016035
3,1,4,0.035248,0.087250,0.041489,0.100189,0.114716,0.138092,0.107025,0.098358,...,0.062605,0.214356,0.192970,0.150780,0.162091,0.207241,0.207771,0.127317,0.121839,0.062577
4,1,5,0.024963,0.082733,0.028976,0.094101,0.119843,0.159927,0.098801,0.101669,...,0.012635,0.187799,0.086104,0.102576,0.198514,0.231088,0.231088,0.185028,0.106626,0.051371


In [ ]:
df_main = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/leaf-classification/leaves.csv')
df_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/leaf-classification/features.csv')

df_features.columns.values[0] = '1'
df_features.columns.values[1] = '1.1'

df_merged = pd.merge(df_main, df_features, on=['1', '1.1'])

In [ ]:
df_merged

,1,1.1,0.72694,1.4742,0.32396,0.98535,1.2,0.83592,0.0046566,0.0039465,...,8101,8102,8103,8104,8105,8106,8107,8108,8109,8110
0,1,2,0.74173,1.5257,0.36116,0.98152,0.99825,0.798670,0.005242,0.005002,...,0.118712,0.214563,0.084731,0.161559,0.168356,0.214563,0.199051,0.144328,0.081321,0.039529
1,1,3,0.76722,1.5725,0.38998,0.97755,1.00000,0.808120,0.007457,0.010121,...,0.008018,0.200710,0.022678,0.158478,0.156798,0.230509,0.133093,0.101094,0.089969,0.016035
2,1,4,0.73797,1.4597,0.35376,0.97566,1.00000,0.816970,0.006877,0.008607,...,0.062605,0.214356,0.192970,0.150780,0.162091,0.207241,0.207771,0.127317,0.121839,0.062577
3,1,5,0.82301,1.7707,0.44462,0.97698,1.00000,0.754930,0.007428,0.010042,...,0.012635,0.187799,0.086104,0.102576,0.198514,0.231088,0.231088,0.185028,0.106626,0.051371
4,1,6,0.72997,1.4892,0.34284,0.98755,1.00000,0.844820,0.004945,0.004451,...,0.054677,0.229000,0.059784,0.159690,0.170883,0.229000,0.197991,0.174899,0.157707,0.075639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,36,6,0.37522,1.1417,0.81725,0.68511,0.58772,0.125230,0.091860,1.535800,...,0.022380,0.215481,0.103017,0.125554,0.215481,0.215481,0.198307,0.199821,0.124882,0.014920
303,36,7,0.28064,1.0849,0.75319,0.72152,0.71404,0.136860,0.078996,1.135800,...,0.087430,0.216148,0.069405,0.126475,0.109320,0.216148,0.216148,0.216148,0.108129,0.110355
304,36,8,0.35344,1.0329,0.78147,0.70737,0.61579,0.135030,0.089763,1.466400,...,0.138484,0.227714,0.091491,0.148124,0.089949,0.227714,0.202591,0.223720,0.067558,0.044418
305,36,9,0.59988,1.1427,0.71532,0.66101,0.47544,0.157470,0.113370,2.339400,...,0.024884,0.215305,0.116264,0.214473,0.213886,0.209298,0.212195,0.186204,0.104681,0.056591


In [ ]:
df_x = df_merged.drop(['1', '1.1'], axis='columns')
df_label = df_merged['1']

In [ ]:
print(df_x.shape)
print(df_label.shape)

(307, 8123)
(307,)


# Dataset PreProcessing

In [ ]:
df_main = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/leaf-classification/leaves.csv')

df_x = df_main.drop(['1', '1.1'], axis='columns')
df_label = df_main['1']

x_train, x_test, y_train, y_test = train_test_split(df_x, df_label, test_size=0.2, shuffle=True, random_state=42)
# x_train, y_train = x_train.to_numpy(), y_train.to_numpy()
# x_test, y_test = x_test.to_numpy(), y_test.to_numpy()

# Scale
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Apply PCA
pca = PCA(n_components=13)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

In [ ]:
print(x_train.shape, type(x_train))
print(x_test.shape, type(x_test))
print(y_train.shape, type(y_train))
print(y_test.shape, type(y_test))

(271, 13) <class 'numpy.ndarray'>
(68, 13) <class 'numpy.ndarray'>
(271,) <class 'pandas.core.series.Series'>
(68,) <class 'pandas.core.series.Series'>


In [ ]:
print(x_train[0])

[-1.26883414 -2.01061902  0.16820206  0.07836161 -0.42196201 -0.40474635
  0.00427277 -0.01143715 -0.04609206  0.11106238 -0.08952961  0.08976784
 -0.07349231]


## KNN Classifier
Key Parameters to Adjust:
*   Number of Neighbors (k):The k parameter defines the number of nearest neighbors to consider for classification. Common practice is to test multiple values of k and select the best performing one using cross-validation.
*   Distance Metric: The distance metric determines how the distance between points is calculated. Common metrics include:
** Euclidean Distance: Standard straight-line distance.
** Manhattan Distance: Sum of absolute differences.
** Minkowski Distance: Generalization of Euclidean and Manhattan.
** Hamming Distance: Used for categorical data.
Cosine Similarity: Measures the cosine of the angle between vectors.
* Weight Function:
** uniform: All points in each neighborhood are weighted equally.
** distance: Points are weighted by the inverse of their distance, so closer neighbors have a greater influence.




In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.2f}")

best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Best Cross-Validation Score: 0.70
Test Accuracy: 0.87


## SVM Classifier
Adjusting parameters for a Support Vector Machine (SVM) involves finding the optimal values for several key hyperparameters to maximize the model's performance.

* C (Regularization Parameter): Controls the trade-off between achieving a low error on the training data and minimizing the norm of the weights.
* Kernel: Determines the type of decision boundary. Common kernels are:
** linear
** poly (polynomial)
** rbf (Radial Basis Function, a.k.a. Gaussian)
** sigmoid
* Gamma: Defines how far the influence of a single training example reaches (only relevant for rbf, poly, and sigmoid kernels).
* Degree: Degree of the polynomial kernel function (if poly is used).

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Only relevant for 'poly' kernel
}

svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.2f}")

best_svc = grid_search.best_estimator_
y_pred = best_svc.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

Best Parameters: {'C': 100, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
Best Cross-Validation Score: 0.74
Test Accuracy: 0.88


## Gaussian Naive Bayes (GaussianNB)
GaussianNB has fewer hyperparameters to tune. The primary parameter is var_smoothing, which adds a small amount of variance to the data to prevent division by zero.


In [ ]:
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

gnb = GaussianNB()
grid_search = GridSearchCV(gnb, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.2f}")

best_gnb = grid_search.best_estimator_
y_pred = best_gnb.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

Best Parameters: {'var_smoothing': 1e-05}
Best Cross-Validation Score: 0.70
Test Accuracy: 0.81


## Decision Tree classifier
Adjusting parameters for a Decision Tree classifier involves tuning several hyperparameters to optimize its performance. The key parameters include:
* criterion: The function to measure the quality of a split. Supported criteria are "gini" for the Gini impurity and "entropy" for the information gain.
* max_depth: The maximum depth of the tree. Limiting the depth of the tree helps control overfitting.
* min_samples_split: The minimum number of samples required to split an internal node.
* min_samples_leaf: The minimum number of samples required to be at a leaf node.
* splitter: The strategy used to choose the split at each node. Supported strategies are "best" to choose the best split and "random" to choose the best random split.

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50, 100],
    'min_samples_split': [2, 10, 20, 50],
    'min_samples_leaf': [1, 5, 10, 20],
    'splitter': ['best', 'random']
}

dt = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.2f}")

best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}
Best Cross-Validation Score: 0.53
Test Accuracy: 0.66


## Random Forest
Key parameters for Random Forest include:
* n_estimators: The number of trees in the forest.
* max_depth: The maximum depth of the tree.
* min_samples_split: The minimum number of samples required to split an internal node.
* min_samples_leaf: The minimum number of samples required to be at a leaf node.
* max_features: The number of features to consider when looking for the best split.
* bootstrap: Whether bootstrap samples are used when building trees.

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 10, 16],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.2f}")

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

Best Parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Cross-Validation Score: 0.77
Test Accuracy: 0.88


## AdaBoost
Key parameters for AdaBoost include:
* n_estimators: The maximum number of estimators at which boosting is terminated.
* learning_rate: Weight applied to each classifier at each boosting iteration.
* base_estimator: The base estimator from which the boosted ensemble is built. The default is DecisionTreeClassifier.

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.1, 1, 10],
    'estimator__max_depth': [1, 2, 3, 4, 5]
}

adb = AdaBoostClassifier(estimator=DecisionTreeClassifier())
grid_search = GridSearchCV(adb, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score:.2f}")

best_adb = grid_search.best_estimator_
y_pred = best_adb.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

Best Parameters: {'estimator__max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 200}
Best Cross-Validation Score: 0.73
Test Accuracy: 0.79


## Neural Network

In [65]:
y_train_encode = tf.keras.utils.to_categorical(y_train)
y_test_encode = tf.keras.utils.to_categorical(y_test)


initial_learning_rate = 0.01
decay_steps = 50
decay_rate = 0.9
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)


model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],), kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.1),
    Dense(37, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(x_train, y_train_encode, epochs=200, batch_size=32,
                    validation_split=0.1, callbacks=[early_stopping])

y_pred = model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_pred.argmax(axis=1))
print(f'Test accuracy: {accuracy}')

Epoch 1/200
8/8 [==============================] - 1s 36ms/step - loss: 2.6998 - accuracy: 0.2881 - val_loss: 2.6268 - val_accuracy: 0.4643
Epoch 2/200
8/8 [==============================] - 0s 9ms/step - loss: 1.3153 - accuracy: 0.6584 - val_loss: 2.2000 - val_accuracy: 0.7500
Epoch 3/200
8/8 [==============================] - 0s 10ms/step - loss: 0.8787 - accuracy: 0.7572 - val_loss: 1.9937 - val_accuracy: 0.6786
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 0.7550 - accuracy: 0.7942 - val_loss: 1.8861 - val_accuracy: 0.6071
Epoch 5/200
8/8 [==============================] - 0s 9ms/step - loss: 0.6897 - accuracy: 0.8189 - val_loss: 1.7637 - val_accuracy: 0.7857
Epoch 6/200
8/8 [==============================] - 0s 7ms/step - loss: 0.6109 - accuracy: 0.8189 - val_loss: 1.6555 - val_accuracy: 0.7500
Epoch 7/200
8/8 [==============================] - 0s 9ms/step - loss: 0.6274 - accuracy: 0.8189 - val_loss: 1.5905 - val_accuracy: 0.8571
Epoch 8/200
8/8 [========

3/3 [==============================] - 0s 5ms/step
Test accuracy: 0.9117646813392639


# Compare Models

In [97]:
classifiers = {
    'KNN': KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance'),
    'SVM': SVC(C=100, kernel='linear', gamma='scale', degree=2),
    'Gaussian Naive Bayes': GaussianNB(var_smoothing=1e-05),
    'Decision Tree': DecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=10),
    'Random Forest': RandomForestClassifier(n_estimators=300, max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, bootstrap=True),
    'AdaBoost': AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), learning_rate=0.1, n_estimators=200)
}

mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],), kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.1),
    Dense(37, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])

optimizer = Adam(learning_rate=0.01)

mlp_model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
mlp_model.fit(x_train, y_train_encode, epochs=200, batch_size=32,
              validation_split=0.1, callbacks=[early_stopping])

y_pred_mlp_probs = mlp_model.predict(x_test)
y_pred_mlp = np.argmax(y_pred_mlp_probs, axis=1)


results = []
conf_matrices = {}
classification_reports = {}

results.append({
    'Classifier': 'MLP',
    'Accuracy': accuracy_score(y_test, y_pred_mlp),
    'Precision': precision_score(y_test, y_pred_mlp, average='weighted'),
    'Recall': recall_score(y_test, y_pred_mlp, average='weighted'),
    'F1 Score': f1_score(y_test, y_pred_mlp, average='weighted'),
})
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrices['MLP'] = conf_matrix



for name, clf in classifiers.items():
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    results.append({
        'Classifier': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1 Score': f1_score(y_test, y_pred, average='weighted'),
    })
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_matrices[name] = conf_matrix

results_df = pd.DataFrame(results)

Epoch 1/200
8/8 [==============================] - 1s 35ms/step - loss: 2.9708 - accuracy: 0.2510 - val_loss: 2.5444 - val_accuracy: 0.5000
Epoch 2/200
8/8 [==============================] - 0s 6ms/step - loss: 1.3113 - accuracy: 0.6132 - val_loss: 2.0903 - val_accuracy: 0.7143
Epoch 3/200
8/8 [==============================] - 0s 9ms/step - loss: 0.9745 - accuracy: 0.7325 - val_loss: 1.9159 - val_accuracy: 0.6786
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 0.8111 - accuracy: 0.7737 - val_loss: 1.7705 - val_accuracy: 0.6786
Epoch 5/200
8/8 [==============================] - 0s 9ms/step - loss: 0.6831 - accuracy: 0.8313 - val_loss: 1.6860 - val_accuracy: 0.7143
Epoch 6/200
8/8 [==============================] - 0s 9ms/step - loss: 0.5924 - accuracy: 0.8519 - val_loss: 1.5589 - val_accuracy: 0.8214
Epoch 7/200
8/8 [==============================] - 0s 9ms/step - loss: 0.5942 - accuracy: 0.8436 - val_loss: 1.4736 - val_accuracy: 0.8571
Epoch 8/200
8/8 [=========

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precis

In [98]:
results_df.sort_values(by='F1 Score', ascending=False, ignore_index=True)

,Classifier,Accuracy,Precision,Recall,F1 Score
0,MLP,0.941176,0.964461,0.941176,0.942577
1,Random Forest,0.882353,0.928922,0.882353,0.886275
2,SVM,0.882353,0.922059,0.882353,0.881559
3,KNN,0.867647,0.922059,0.867647,0.874767
4,AdaBoost,0.823529,0.887255,0.823529,0.831956
5,Gaussian Naive Bayes,0.808824,0.862745,0.808824,0.805952
6,Decision Tree,0.647059,0.658824,0.647059,0.615920
